**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 23.5MB/s 


In [4]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Let's consider the agent in 2 possible states of mind :
- Training (when train=True)
- Applying (when train=False)

The function `act` does the following:
- If the agent is training, in $100\cdot\epsilon$ % cases, it will explore the environment by randomly chosing its next action, else it will apply the best action it learned at this current environment state
- If the agent is applying, it will use what it learned in the training step and for each state, apply the best action it learned.

`epsilon` is essential because it handles how much we want our agent to explore all possible policies. If it is equal to $0$, then the agent will look at one possible action for each state and thus, will not learn the optimal policy. $\epsilon$ is a kind of tradeoff between "keeping my knowledge" (exploitation) and "discovering other possibilites" (exploration).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4 # Add border
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :]  = 0
        b[:, :2, :]  = 0
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t, :, :, :] = b

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1

        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t += 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                                axis=2)
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus > 0] = 0
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),
                                axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 101 # set small when debugging
epochs_test = 11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Both `position` and `board` arrays are of shape `(grid_size, grid_size)`.

`board` array contain all information about the cells of the game, where are the cheeses (value: 0.5) and poisons (value: -1). Each cell becomes equal to $0$ when the rat comes over it. 

`position` array contains zeros everywhere except where the rat is (it is equal to one there).

The concatenation of these 2 arrays is the game state tensor.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # We return a random action
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/16.0. Average score (-5.5)
Win/lose count 13.5/18.0. Average score (-5.0)
Win/lose count 13.0/15.0. Average score (-4.0)
Win/lose count 10.5/21.0. Average score (-5.625)
Win/lose count 10.0/16.0. Average score (-5.7)
Win/lose count 14.5/12.0. Average score (-4.333333333333333)
Win/lose count 9.0/10.0. Average score (-3.857142857142857)
Win/lose count 6.0/10.0. Average score (-3.875)
Win/lose count 10.0/15.0. Average score (-4.0)
Win/lose count 9.0/16.0. Average score (-4.3)
Win/lose count 13.0/11.0. Average score (-3.727272727272727)
Final score: -3.727272727272727


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**:

<u>Proof 1:</u>

We have :
$$ Q^{\pi}(s, a) = E_{p^{\pi}} \left[ \sum_{t \leq T} \gamma^t r(s_t, a_t) | s_0=s, a_0=a \right] $$

Hence :
$$ Q^{\pi}(s, a) = E_{p^{\pi}} \left[ r(s_0, a_0) | s_0=s, a_0=a \right] + E_{p^{\pi}} \left[ \sum_{0 < t \leq T} \gamma^t r(s_t, a_t) | s_0=s, a_0=a \right] $$

$$ Q^{\pi}(s, a) = r(s, a) + \gamma E_{p^{\pi}} \left[ \sum_{0 < t \leq T} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a \right]$$

$$ Q^{\pi}(s, a) = r(s, a) + \gamma E_{(s', a') \sim p^{\pi}(\cdot|s,a)} \left[ E_{p^{\pi}} \left[ \sum_{0 \leq t \leq T-1} \gamma^{t} r(s_t, a_t) | s_0=s', a_0=a' \right] \right]$$

$$ Q^{\pi}(s, a) = r(s, a) + \gamma E_{(s', a') \sim p^{\pi}(\cdot|s,a)} \left[ Q^{\pi}(s', a') \right]$$

Finally:

$$\boxed{Q^{\pi}(s, a) = E_{(s', a') \sim p^{\pi}(\cdot|s,a)} \left[ r(s, a) + \gamma Q^{\pi}(s', a') \right]}$$

<u>Proof 2:</u>

Then we have:

$$ Q^{*}(s,a) = \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$

$$ Q^{*}(s,a) = \max_{\pi} r(s, a) + \gamma \sum p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a') $$

$$ Q^{*}(s,a) = r(s,a) + \gamma \max_{\pi} \sum p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \qquad \text{with } \pi=(a',\pi') $$

We can exchange the maximum and the sum with the following inequality :

$$ \max_{\pi} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \leq \sum_{(s',a')} p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi}Q^{\pi'}(s',a') $$

Then we have : (considering $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$)

$$
\begin{align*}
 \sum_{s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi'}Q^{\pi'}(s',a') &= \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\hat{\pi}(s',a')}(s',a') \\
 &\leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') 
\end{align*}
$$

So :

$$ Q^{*}(s,a) = r(s,a) + \gamma  \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') $$


$$ Q^{*}(s,a) = r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') $$

Finally:

$$ \boxed{Q^{*}(s, a) = E_{s' \sim \pi^{*}(\cdot|s,a)} \left[ r(s, a) + \underset{a'}{\max} \gamma Q^{*}(s', a') \right]}$$

<u>Proof 3:</u>

Let $Q^{\theta}(s, a) = Q(s, a, \theta)$ be our $Q$ function (approx. by a neural network). We want $Q^{\theta} \approx Q^{*}$. In other terms, $Q^{\theta}$ should verify :

$$ Q^{\theta}(s, a) = E_{s'\sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') \right] $$

$$ E_{s'\sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') - Q^{\theta}(s, a) \right] = 0 $$

$$ \left|\left| E_{s'\sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') - Q^{\theta}(s, a) \right] \right|\right|^2 = 0 $$

Hence :

$$ E_{s'\sim \pi^*(.|s,a)} \left|\left| r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') - Q^{\theta}(s, a) \right|\right|^2 = 0 $$

So, we can define an objective function :

$$ \boxed{\mathcal{L}(\theta) = E_{s'\sim \pi^*(.|s,a)} \left|\left| r + \gamma\max_{a'}Q(s', a', \theta) - Q(s, a, \theta) \right|\right|^2} $$

that has to be minimized.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory:
            self.memory.pop(0)
        self.memory.append(m)

    def random_access(self):
        index = np.random.randint(0, len(self.memory))
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q_values = self.model.predict(np.array([s]))[0]
        return np.argmax(q_values)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        if len(self.memory.memory) < self.batch_size:
            return 0.0
        
        minibatch = [self.memory.random_access() for i in range(self.batch_size)]

        input_states = np.array([values[0] for values in minibatch])
        input_qs_list = self.model.predict(input_states)

        new_states = np.array([values[1] for values in minibatch])
        future_qs_list = self.model.predict(new_states)

        target_q = np.zeros((self.batch_size, 4))
        
        for i, (current_state, new_current_state, action, reward, game_over) in enumerate(minibatch):
            if game_over:
                target_q_value = reward
            else:
                target_q_value = reward + self.discount * np.max(future_qs_list[i])
              
            target_q[i] = input_qs_list[i]
            target_q[i, action] = target_q_value

        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class DQN_FC(DQN):
    def __init__(self, lr=0.1, *args, **kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        # NN Model

        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        flatten = keras.layers.Flatten(name="flatten")(input_model)
        fc1 = Dense(100, name="fc1", activation="relu")(flatten)
        fc2 = Dense(100, name="fc2", activation="relu")(fc1)
        fc3 = Dense(100, name="fc3", activation="relu")(fc2)
        output_model = Dense(4, name="output")(fc3)
        model = keras.models.Model(input_model, output_model)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=.1, grid_size=size, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train100.mp4'))

Epoch 000/101 | Loss 0.0018 | Win/lose count 1.5/0 (1.5)
Epoch 001/101 | Loss 0.0009 | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/101 | Loss 0.0056 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/101 | Loss 0.0006 | Win/lose count 6.5/11.0 (-4.5)
Epoch 004/101 | Loss 0.0039 | Win/lose count 2.5/5.0 (-2.5)
Epoch 005/101 | Loss 0.0049 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/101 | Loss 0.0142 | Win/lose count 4.0/5.0 (-1.0)
Epoch 007/101 | Loss 0.0027 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/101 | Loss 0.0052 | Win/lose count 3.5/5.0 (-1.5)
Epoch 009/101 | Loss 0.0263 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/101 | Loss 0.0022 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/101 | Loss 0.0149 | Win/lose count 5.5/5.0 (0.5)
Epoch 012/101 | Loss 0.0449 | Win/lose count 6.5/8.0 (-1.5)
Epoch 013/101 | Loss 0.0268 | Win/lose count 4.5/2.0 (2.5)
Epoch 014/101 | Loss 0.0036 | Win/lose count 3.5/3.0 (0.5)
Epoch 015/101 | Loss 0.0062 | Win/lose count 1.5/3.0 (-1.5)
Epoch 016/101 | Loss 0.0024 | Win/lose count 3.5/1

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        cn1_1 = Conv2D(32, 3, padding="same", activation="relu", name="cn1_1")(input_model)
        cn1_2 = Conv2D(32, 3, padding="same", activation="relu", name="cn1_2")(cn1_1)
        mp1 = MaxPooling2D(name="mp1")(cn1_2)
        cn2_1 = Conv2D(64, 3, padding="same", activation="relu", name="cn2_1")(mp1)
        cn2_2 = Conv2D(64, 3, padding="same", activation="relu", name="cn2_2")(cn2_1)
        flatten = keras.layers.Flatten(name="flatten")(cn2_2)
        output_model = Dense(4, name="output")(flatten)
        model = keras.models.Model(input_model, output_model)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [21]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size=size, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train100.mp4'))


Epoch 000/101 | Loss 0.0001 | Win/lose count 1.5/2.0 (-0.5)
Epoch 001/101 | Loss 0.0115 | Win/lose count 2.5/10.0 (-7.5)
Epoch 002/101 | Loss 0.0002 | Win/lose count 3.5/5.0 (-1.5)
Epoch 003/101 | Loss 0.0076 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/101 | Loss 0.0120 | Win/lose count 1.5/4.0 (-2.5)
Epoch 005/101 | Loss 0.0166 | Win/lose count 2.0/5.0 (-3.0)
Epoch 006/101 | Loss 0.0071 | Win/lose count 1.0/2.0 (-1.0)
Epoch 007/101 | Loss 0.0019 | Win/lose count 2.0/1.0 (1.0)
Epoch 008/101 | Loss 0.0017 | Win/lose count 1.0/2.0 (-1.0)
Epoch 009/101 | Loss 0.0109 | Win/lose count 6.0/2.0 (4.0)
Epoch 010/101 | Loss 0.0065 | Win/lose count 3.0/4.0 (-1.0)
Epoch 011/101 | Loss 0.0001 | Win/lose count 3.5/1.0 (2.5)
Epoch 012/101 | Loss 0.0001 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/101 | Loss 0.0097 | Win/lose count 3.5/4.0 (-0.5)
Epoch 014/101 | Loss 0.0104 | Win/lose count 1.5/6.0 (-4.5)
Epoch 015/101 | Loss 0.0037 | Win/lose count 3.0/2.0 (1.0)
Epoch 016/101 | Loss 0.0027 | Win/lose count

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
# Load agents
agent_cnn = DQN_CNN(lr=.1, grid_size=size, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(lr=.1, grid_size=size, epsilon=0.1, memory_size=2000, batch_size=32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

In [0]:
# Load env
env = Environment(grid_size=size, max_time=T, temperature=0.3)

In [24]:
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

Test of the FC
Win/lose count 5.5/2.0. Average score (3.5)
Win/lose count 6.5/1.0. Average score (4.5)
Win/lose count 11.5/1.0. Average score (6.5)
Win/lose count 11.5/4.0. Average score (6.75)
Win/lose count 4.5/0. Average score (6.3)
Win/lose count 9.0/4.0. Average score (6.083333333333333)
Win/lose count 6.0/0. Average score (6.071428571428571)
Win/lose count 15.0/4.0. Average score (6.6875)
Win/lose count 5.5/1.0. Average score (6.444444444444445)
Win/lose count 9.5/1.0. Average score (6.65)
Win/lose count 10.5/4.0. Average score (6.636363636363637)
Final score: 6.636363636363637
Test of the CNN
Win/lose count 9.5/4.0. Average score (5.5)
Win/lose count 14.5/1.0. Average score (9.5)
Win/lose count 12.0/0. Average score (10.333333333333334)
Win/lose count 15.5/0. Average score (11.625)
Win/lose count 15.0/4.0. Average score (11.5)
Win/lose count 10.0/3.0. Average score (10.75)
Win/lose count 15.0/2.0. Average score (11.071428571428571)
Win/lose count 12.0/3.0. Average score (10.8125

In [25]:
HTML(display_videos('fc_test10.mp4'))

In [29]:
HTML(display_videos('cnn_test10.mp4'))

The performance using the architecture based on CNN seems better than the one obtained using some fully connected networks.

In [0]:
# Load env
env = Environment(grid_size=size, max_time=T, temperature=0.8)

In [30]:
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

Test of the FC
Win/lose count 44.0/3.0. Average score (41.0)
Win/lose count 37.0/2.0. Average score (38.0)
Win/lose count 53.0/3.0. Average score (42.0)
Win/lose count 32.0/2.0. Average score (39.0)
Win/lose count 31.5/1.0. Average score (37.3)
Win/lose count 32.0/0. Average score (36.416666666666664)
Win/lose count 57.5/0. Average score (39.42857142857143)
Win/lose count 40.5/3.0. Average score (39.1875)
Win/lose count 17.5/3.0. Average score (36.44444444444444)
Win/lose count 51.5/3.0. Average score (37.65)
Win/lose count 46.0/4.0. Average score (38.04545454545455)
Final score: 38.04545454545455
Test of the CNN
Win/lose count 28.5/0. Average score (28.5)
Win/lose count 41.5/0. Average score (35.0)
Win/lose count 41.5/4.0. Average score (35.833333333333336)
Win/lose count 34.0/2.0. Average score (34.875)
Win/lose count 31.0/3.0. Average score (33.5)
Win/lose count 44.5/1.0. Average score (35.166666666666664)
Win/lose count 49.5/4.0. Average score (36.642857142857146)
Win/lose count 44

In [31]:
HTML(display_videos('fc_test10.mp4'))

In [32]:
HTML(display_videos('cnn_test10.mp4'))

The gap between their performances is decreasing with the temperature. This means that the CNN version is more able to reach cheese even when they are quite rare.

The main issue that we can observe is that the algorithms tends to not explore the map that means that sometimes it goes to a position A then goes to a position B and then go back to position A and so on ...

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    # init. epsilon
    decreasing_factor = 0.9

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        # Update epsilon
        agent.set_epsilon(agent.epsilon * decreasing_factor)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4 # Add border
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :]  = 0
        b[:, :2, :]  = 0
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t, :, :, :] = b

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1

        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t += 1
        reward = (0 if not train else -self.malus_position[self.x, self.y]) + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        self.malus_position[self.x, self.y] += 0.1
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                                axis=2)
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus > 0] = 0
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.board[self.x,self.y] = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                                axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state

In [40]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size=size, epsilon=0.5, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

Epoch 000/101 | Loss 0.0104 | Win/lose count 12.0/48.000000000000014 (-36.000000000000014)
Epoch 001/101 | Loss 0.0226 | Win/lose count 11.0/49.60000000000003 (-38.60000000000003)
Epoch 002/101 | Loss 0.0252 | Win/lose count 12.5/37.89999999999999 (-25.39999999999999)
Epoch 003/101 | Loss 0.0260 | Win/lose count 12.5/36.800000000000004 (-24.300000000000004)
Epoch 004/101 | Loss 0.0111 | Win/lose count 17.5/29.70000000000002 (-12.20000000000002)
Epoch 005/101 | Loss 0.0122 | Win/lose count 20.5/24.699999999999996 (-4.199999999999996)
Epoch 006/101 | Loss 0.0519 | Win/lose count 16.0/33.29999999999999 (-17.29999999999999)
Epoch 007/101 | Loss 0.0335 | Win/lose count 18.5/21.400000000000016 (-2.9000000000000163)
Epoch 008/101 | Loss 0.0300 | Win/lose count 19.5/32.10000000000004 (-12.600000000000037)
Epoch 009/101 | Loss 0.0330 | Win/lose count 26.0/23.60000000000002 (2.399999999999981)
Epoch 010/101 | Loss 0.0157 | Win/lose count 24.0/22.300000000000008 (1.6999999999999922)
Epoch 011/101

In [41]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 25.0/6.0. Average score (19.0)
Win/lose count 21.0/7.0. Average score (16.5)
Win/lose count 21.5/7.0. Average score (15.833333333333334)
Win/lose count 26.5/5.0. Average score (17.25)
Win/lose count 22.0/8.0. Average score (16.6)
Win/lose count 21.5/7.0. Average score (16.25)
Win/lose count 20.5/10.0. Average score (15.428571428571429)
Win/lose count 16.0/15.0. Average score (13.625)
Win/lose count 25.5/4.0. Average score (14.5)
Win/lose count 25.0/10.0. Average score (14.55)
Win/lose count 17.5/7.0. Average score (14.181818181818182)
Final score: 14.181818181818182


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***